# Source

The `Source` class provides an interface and predefined methods for reading raw data from file.
If `source.use_h5 == True` the raw data will be saved and read preferentially from a generated `hdf5` file, which results in a single data file with high compression and fast data access.

The `Source` class provides a `dict()` of all scans, called `scan_dict`.
The items in this dictionary are instances of the `Scan` class, which defines the common data structure.
However, the access to the scan data should NOT be done via the `scan_dict` directly, but instead for a deticated scan via `source.get_scan_data(scan_number)` or for a list of scans via `source.get_scan_list_data(scan_number_list)`.
By this all features of the `Source` class can be provided.

One key concept of the `pyEvalData` is the idea, that `Scan` instance must only provide the meta information of the scan but necessarily also its data.
By the default, the scan data will only be read on request and is then stored in the `Scan` object for later use.
It is possible, however to read all data to each scan by default by `source.read_all_data = Ture`.
On the other hand, if the data is allocating too much memory, it is possible to clear the data from the `Scan` object, directly after accessing it, via `source.read_and_forget = True`.
The flag `source.update_before_read` enables parsing the raw source file to search and add new scans before accessing any scan.
By default the last `Scan` in the `scan_dict` will always be re-created in case new data was added.
The flag `source.force_overwrite` will force a full parsing of the raw source file and a complete overwrite of the `hdf5` file.

Any specific data source class must inherit from the base `Source` class and must only provide the two methods `parse_raw()` and `read_raw_ scan_data(scan)`, which parse the raw source to create the `scan_dict` and to read the actual scan data, respectivly.
With that a rather generic interface for nearly any kind of data source should be possible.

## Setup

Here we do the necessary import for this example

In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np

import pyEvalData as ped

h5py._conv - DEBUG: Creating converter from 7 to 5
h5py._conv - DEBUG: Creating converter from 5 to 7
h5py._conv - DEBUG: Creating converter from 7 to 5
h5py._conv - DEBUG: Creating converter from 5 to 7
cannot open database file c:\users\loc_schick\general\git\xrayutilities\lib\xrayutilities\materials\data\elements.db!


# SPEC

The most common data source are SPEC files from the original [Certif SPEC](https://certif.com/content/spec) as well as from the open-source alternative [Sardana](https://sardana-controls.org).
The `Spec` source relies on the great parser provided by [xrayutilities](https://xrayutilities.sourceforge.io).

In [4]:
spec = ped.io.Spec(file_name='certif_xu.spec', file_path='data/')

pyEvalData.io.source - INFO: Update source
pyEvalData.io.source - DEBUG: Updating from h5
pyEvalData.io.source - DEBUG: Update before read
pyEvalData.io.source - DEBUG: parse_raw
pyEvalData.io.scan - DEBUG: Creating scan #33
pyEvalData.io.scan - DEBUG: Creating scan #34
pyEvalData.io.scan - DEBUG: Creating scan #35
pyEvalData.io.scan - DEBUG: Creating scan #36
pyEvalData.io.source - DEBUG: save_all_scans_to_h5
pyEvalData.io.source - DEBUG: save_scan_to_h5 scan #33
pyEvalData.io.source - DEBUG: save_scan_to_h5 scan #34
pyEvalData.io.source - DEBUG: save_scan_to_h5 scan #35
pyEvalData.io.source - DEBUG: save_scan_to_h5 scan #36


## Composite Sources

On specific idea of the `Source` class is to provide also composite models of pre-defined `Source` classes.
An example is the `Spec` source for reading SPEC data files and a `Source` class to read camera images from a folder structure, that have been acquired simultaneously with the SPEC file.
This would require to specify the two independent `Souce` objects and then add them to a single `CompositeSource`:
```python
spec = ed.io.Spec(file_name, file_path)
ccd = ed.io.Pilatus(image_pattern, image_base_path)

comp_source = spec + ccd
```